# TCC 1
* Aluno: Paulo Henrique Costa Gontijo
* Matrícula: 15/0143800


In [1]:
# !pip install pyspark fastai fastbook -q

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# # from google.colab import files
# #
# # files.upload() #this will prompt you to upload the kaggle.json
#
# !ls -lha kaggle.json
#
# # Install kaggle API client
# !pip install -q kaggle
#
# # kaggle API client expects the file to be in ~/.kaggle
#
# # so move it there
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
#
# # we need to set permissions
# !chmod 600 /root/.kaggle/kaggle.json
#
# # check your directory before downloading the datasets
# !pwd
#
# # # list all available datasets
# # !kaggle datasets list
#
# # download the required dataset from kaggle
# !kaggle competitions download -c otto-recommender-system

# # If your file is a zip file you can unzip with the following code
# !unzip /content/otto-recommender-system.zip

In [4]:
# !unzip ./otto-recommender-system.zip

## Importação das Bibliotecas

In [5]:
import os
import pandas as pd

file_name = 'test.jsonl'
file_path = '/'.join([os.getcwd(), file_name])

## Criando Cluster Spark

In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from os.path import abspath

warehouse_location = abspath('spark-warehouse')

    # init session
# spark = SparkSession \
#         .builder \
#         .appName("otto-reccomender-competition-kaggle") \
#         .config("spark.sql.warehouse.dir", abspath('spark-warehouse')) \
#         .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
#         .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")\
#         .config("spark.sql.shuffle.partitions", "10000")\
#         .config("spark.driver.maxResultSize", "16g")\
#         .config("spark.driver.memory", "16g")\
#         .config("spark.driver.memoryOverheadFactor", "0.30")\
#         .config("spark.executor.pyspark.memory", "32g")\
#         .config("spark.checkpoint.compress", "true")\
#         .enableHiveSupport() \
#         .getOrCreate()

spark = SparkSession \
        .builder \
        .appName("otto-reccomender-competition-kaggle") \
        .config("spark.sql.warehouse.dir", abspath('spark-warehouse')) \
        .config("spark.executor.cores", "10")\
        .config("spark.driver.cores", "10")\
        .config("spark.executor.instances", "2")\
        .config("spark.executor.memory","32g")\
        .config("spark.driver.memory", "32g")\
        .config("spark.default.parallelism","80")\
        .config("spark.sql.shuffle.partitions","20")\
        .config("spark.sql.autoBroadcastHashJoin","-1")\
        .config("spark.sql.files.maxPartitionBytes","1g")\
        .enableHiveSupport() \
        .getOrCreate()

# show configured parameters
print(SparkConf().getAll())

# set log level
spark.sparkContext.setLogLevel("FATAL")

23/09/26 10:26:16 WARN Utils: Your hostname, gobellek-B660M-DS3H-DDR4 resolves to a loopback address: 127.0.1.1; using 192.168.0.37 instead (on interface enp4s0)
23/09/26 10:26:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/26 10:26:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.sql.shuffle.partitions', '20'), ('spark.app.submitTime', '1695734777168'), ('spark.driver.memory', '32g'), ('spark.driver.cores', '10'), ('spark.default.parallelism', '80'), ('spark.executor.cores', '10'), ('spark.executor.memory', '32g'), ('spark.app.name', 'otto-reccomender-competition-kaggle'), ('spark.sql.autoBroadcastHashJoin', '-1'), ('spark.sql.catalogImplementation', 'hive'), ('spark.executor.instances', '2'), ('spark.sql.warehouse.dir', '/home/gobellek/Documents/UnB/Tcc/tcc/spark-warehouse'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.sql.files.maxPartitionBytes', '1g')]


* Conferindo o instanciamento

In [7]:
spark

* Leitura do arquivo disponibilizado na competição kaggle: [link.](https://www.kaggle.com/competitions/otto-recommender-system/overview/)

In [8]:
df_raw = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .json(file_path).persist()
df_raw.columns

['events', 'session']

* Conferindo número de partições e conteúdo

In [9]:
print(df_raw.rdd.getNumPartitions())

80


In [10]:
df_raw.show(10)

+--------------------+--------+
|              events| session|
+--------------------+--------+
|[{59625, 16617240...|12899779|
|[{1142000, 166172...|12899780|
|[{141736, 1661724...|12899781|
|[{1669402, 166172...|12899782|
|[{255297, 1661724...|12899783|
|[{1036375, 166172...|12899784|
|[{1784451, 166172...|12899785|
|[{955252, 1661724...|12899786|
|[{1682750, 166172...|12899787|
|[{245131, 1661724...|12899788|
+--------------------+--------+
only showing top 10 rows



* Contagem de linhas em arquivo bruto

In [11]:
df_raw.count()

1671803

In [12]:
df_raw.printSchema()

root
 |-- events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- aid: long (nullable = true)
 |    |    |-- ts: long (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- session: long (nullable = true)



* Processamento de Json para formato tabular

In [13]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [14]:
def explode_df_json_to_tabular(df_raw):
    df_explode = df_raw.withColumn('events_explode', F.explode('events'))\
                    .withColumn('session', F.col('session').cast(T.IntegerType()))\
                    .withColumn('aid', F.col('events_explode.aid').cast(T.IntegerType()))\
                    .withColumn('ts', F.col('events_explode.ts'))\
                    .withColumn('type', F.col('events_explode.type'))\
                    .drop('events', 'events_explode')
    return df_explode

In [15]:
df_explode = explode_df_json_to_tabular(df_raw)
df_explode.show(10)

+--------+-------+-------------+------+
| session|    aid|           ts|  type|
+--------+-------+-------------+------+
|12899779|  59625|1661724000278|clicks|
|12899780|1142000|1661724000378|clicks|
|12899780| 582732|1661724058352|clicks|
|12899780| 973453|1661724109199|clicks|
|12899780| 736515|1661724136868|clicks|
|12899780|1142000|1661724155248|clicks|
|12899781| 141736|1661724000559|clicks|
|12899781| 199008|1661724022851|clicks|
|12899781|  57315|1661724170835|clicks|
|12899781| 194067|1661724246188|clicks|
+--------+-------+-------------+------+
only showing top 10 rows



* Contagem de linhas após o tratamento

In [16]:
df_explode.count()

6928123

* Schema pós tratamento

In [17]:
df_explode.printSchema()

root
 |-- session: integer (nullable = true)
 |-- aid: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- type: string (nullable = true)



* Categorização de coluna alvo string

In [18]:
from pyspark.ml.feature import StringIndexer

def transform_indexer(df_explode):
    indexer = StringIndexer(inputCol='type', outputCol='type_cat')
    indexer_fitted = indexer.fit(df_explode)
    df_indexed = indexer_fitted.transform(df_explode)
    df_indexed_type = df_indexed.withColumn('type_cat', F.col('type_cat').cast(T.IntegerType())).drop('type')
    df_indexed_type_renamed = df_indexed_type.withColumnRenamed('session','userCol')
    df_indexed_droped = df_indexed_type_renamed.dropDuplicates()
    df = df_indexed_droped.drop('ts')
    return df

In [19]:
df_indexed = transform_indexer(df_explode).persist()
df_indexed.show(10)

+--------+-------+--------+
| userCol|    aid|type_cat|
+--------+-------+--------+
|12899780| 582732|       0|
|12899782| 562753|       1|
|12899782| 834354|       2|
|12899783|1216820|       0|
|12899785|1784451|       0|
|12899785| 903397|       0|
|12899785|1308865|       0|
|12899785| 614626|       0|
|12899788|1663048|       0|
|12899791|1787713|       0|
+--------+-------+--------+
only showing top 10 rows



* Conferência de schema pós tratativas

In [20]:
df_indexed.printSchema()

root
 |-- userCol: integer (nullable = true)
 |-- aid: integer (nullable = true)
 |-- type_cat: integer (nullable = true)



## Análise de Clientes

* Quantidade de clientes únicos

In [21]:
df_indexed.select('userCol').distinct().count()

1671803

## Análise de Produtos

* Quantidade de produtos únicos

In [22]:
df_indexed.select('aid').distinct().count()

783486

# Escrita de DataFrame tratado

In [23]:
df_indexed.coalesce(20)\
          .write.format('parquet')\
          .mode("overwrite")\
          .save('/'.join([os.getcwd(), 'refined-dataset']))

In [24]:
df_raw.unpersist()
df_explode.unpersist()
df_indexed.unpersist()

DataFrame[userCol: int, aid: int, type_cat: int]

# Lendo o dataset refinado

In [25]:
df_refined = spark.read.format('parquet').load('refined-dataset').persist()
df_refined.show()

+--------+-------+--------+
| userCol|    aid|type_cat|
+--------+-------+--------+
|12899781| 199008|       0|
|12899782| 975116|       0|
|12899782| 975116|       1|
|12899782| 889671|       1|
|12899785|1154962|       0|
|12899785| 453905|       0|
|12899785| 258458|       0|
|12899785|1553332|       0|
|12899785| 874493|       0|
|12899785|  10851|       0|
|12899785| 453905|       0|
|12899799| 413826|       2|
|12899801| 725164|       0|
|12899803| 795006|       0|
|12899803| 929024|       1|
|12899803|1651971|       0|
|12899803|1684753|       2|
|12899807|1407421|       0|
|12899809| 703890|       0|
|12899810|1309871|       0|
+--------+-------+--------+
only showing top 20 rows



# Filtrando Dataset de Treino para Clientes que Compraram

In [26]:
# df_pre_moodel = df_refined.select(df_refined.columns).where(F.col('type_cat') == 2)
df_pre_moodel = df_refined
df_pre_moodel.show()

+--------+-------+--------+
| userCol|    aid|type_cat|
+--------+-------+--------+
|12899781| 199008|       0|
|12899782| 975116|       0|
|12899782| 975116|       1|
|12899782| 889671|       1|
|12899785|1154962|       0|
|12899785| 453905|       0|
|12899785| 258458|       0|
|12899785|1553332|       0|
|12899785| 874493|       0|
|12899785|  10851|       0|
|12899785| 453905|       0|
|12899799| 413826|       2|
|12899801| 725164|       0|
|12899803| 795006|       0|
|12899803| 929024|       1|
|12899803|1651971|       0|
|12899803|1684753|       2|
|12899807|1407421|       0|
|12899809| 703890|       0|
|12899810|1309871|       0|
+--------+-------+--------+
only showing top 20 rows



In [27]:
df_pre_moodel.select('userCol').distinct().count()

1671803

In [28]:
df_pre_moodel.count()

6924640

### Adicionar pipeline de hiperparametrização
* http://restanalytics.com/2019-02-27-Hyperparameter-Tuning-Alternating-Least-Squares-Recommender-System/

In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# (training, test) = df_pre_moodel.randomSplit([0.6, 0.4])
training = df_pre_moodel

In [30]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=10,
          maxIter=20,
          regParam=0.01,
          userCol="userCol",
          itemCol="aid",
          ratingCol="type_cat",
          coldStartStrategy="drop",
          implicitPrefs=True,
          checkpointInterval=1)
model = als.fit(training)

In [31]:
# # Evaluate the model by computing the RMSE on the test data
# predictions = model.transform(test)
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="type_cat",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

# Utilizando dataset de teste da competição

In [ ]:
file_name = 'test.jsonl'
file_path = '/'.join([os.getcwd(), file_name])

In [ ]:
df_test = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .json(file_path).persist()

In [ ]:
df_test.show()

In [ ]:
df_test.count()

# Tratamento de subset para recomendações

In [ ]:
df_pre_transform = explode_df_json_to_tabular(df_test)
df_pre_transform.count()

In [ ]:
df_subset = transform_indexer(df_pre_transform).select('userCol').distinct()
df_subset.count()

In [ ]:
df_pre_moodel.join(df_subset, df_pre_moodel.userCol == df_subset.userCol, 'inner')\
                    .drop(df_subset.userCol).select('userCol').distinct().count()

In [ ]:
df_recommend = model.recommendForUserSubset(df_subset, 20)
df_recommend.printSchema()

# Escrevendo Dataset de Recomendações em JSON

In [ ]:
df_recommend.write.format('json')\
                  .mode("overwrite")\
                  .save('/'.join([os.getcwd(), 'raw_predictions.json']))

In [ ]:
df_recommend.unpersist()

# Leitura de Dataset RAW_PREDICTIONS
Nesta etapa será realizada a conversão do objeto JSON em formato tabular, para se adequar ao formato de submissão da competição.

In [ ]:
df_recommend_json = spark.read.format('json').load('./raw_predictions.json').persist()
df_recommend_json.printSchema()

In [ ]:
df_recommend_json.show(10, truncate=False)

* Tratamento JSON->Tabular

In [ ]:
df_recommend_json_exploded = df_recommend_json.withColumn('recommendations_explode', F.explode('recommendations'))\
                                    .withColumn('aid', F.col('recommendations_explode.aid').cast(T.IntegerType()))\
                                    .drop('recommendations_explode', 'recommendations')
df_recommend_json_exploded.printSchema()

In [ ]:
df_recommend_json_exploded.show(10)

# Agrupando top 20 itens por usuário

In [ ]:
df_submission_raw = df_recommend_json_exploded.withColumn('userCol', F.col('userCol').cast(T.StringType()))\
                                                .groupby('userCol').agg(F.collect_list('aid').alias("labels")).persist()
df_submission_raw = df_submission_raw.withColumnRenamed('userCol', 'session_type')
df_submission_raw.show(10, truncate=False)

In [ ]:
df_submission_raw.count()

# Criando dataset no formato de submissão

In [ ]:
users_list_raw = [r[0] for r in df_submission_raw.select('session_type').toLocalIterator()]
users_list_raw[:5]

In [ ]:
users_list_clicks = [s + '_clicks' for s in users_list_raw]
users_list_clicks[:5]

In [ ]:
users_list_carts = [s + '_carts' for s in users_list_raw]
users_list_carts[:5]

In [ ]:
users_list_orders = [s + '_orders' for s in users_list_raw]
users_list_orders[:5]

In [ ]:
labels_list = [r[0] for r in df_submission_raw.select('labels').toLocalIterator()]
labels_list[:5]

In [ ]:
df_clicks = spark.createDataFrame(zip(users_list_clicks, labels_list), ['session_type', 'labels'])
df_clicks.show(5)

In [ ]:
df_carts = spark.createDataFrame(zip(users_list_carts, labels_list), ['session_type', 'labels'])
df_carts.show(5)

In [ ]:
df_orders = spark.createDataFrame(zip(users_list_orders, labels_list), ['session_type', 'labels'])
df_orders.show(5)

In [ ]:
from functools import reduce
from pyspark.sql import DataFrame

dfs = [df_clicks,df_carts,df_orders]
df_submission = reduce(DataFrame.unionAll, dfs).persist()
df_submission = df_submission.withColumn("labels",F.concat_ws(" ",F.col("labels")))
df_submission.show()

In [ ]:
df_submission.count()

# Escrevendo dataset de submissão

In [ ]:
df_submission.orderBy('session_type').coalesce(1)\
                                     .write.format('csv')\
                                     .mode("overwrite")\
                                     .option("header", True)\
                                     .save('/'.join([os.getcwd(), 'submission']))

# Referências Bibliográficas

> https://github.com/Kaggle/kaggle-api, acessado em 24/01/2023.

> https://www.youtube.com/watch?v=aBNQzWV_UmE, acessado em 24/01/2023.

> https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/968100988546031/157591980591166/8836542754149149/latest.html, acessado em 25/01/2023.

> https://blog.clairvoyantsoft.com/apache-spark-out-of-memory-issue-b63c7987fff, acessado em 25/01/2023.

> https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_ps.html, acessado em 26/01/2023.

> https://docs.fast.ai/collab.html, acessado em 24/01/2023.

> https://course.fast.ai/Lessons/lesson7.html, acessado em 24/01/2023.

> https://github.com/fastai/fastbook/blob/master/08_collab.ipynb, acessado em 24/01/2023.

> https://www.kaggle.com/code/vchulski/tutorial-collaborative-filtering-with-pyspark

> https://medium.com/analytics-vidhya/crafting-recommendation-engine-in-pyspark-a7ca242ad40a

> https://www.freecodecamp.org/news/8-clustering-algorithms-in-machine-learning-that-all-data-scientists-should-know/

> https://www.youtube.com/watch?v=fpqa0_U4zb4

> https://www.youtube.com/watch?v=58OjaDH2FI0&t=742s

> https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

> https://spark.apache.org/docs/latest/configuration.html#memory-management